In [1]:
import sys
print(sys.executable)

c:\Users\basil\Anaconda\python.exe


In [2]:
import sys
sys.path.append('../modules')  # go up one folder, then into "modules"

import utils
utils.configure_plotly_template(showlegend=True)

In [3]:
import os
os.getcwd()

'c:\\Users\\basil\\Documents\\Time-Series-Forecasting-Python-\\5_Baseline'

In [10]:
import pandas as pd
df = pd.read_parquet('../statsmodel/AirPassengers.parquet')
df

,Passengers
1949-01-31,112
1949-02-28,118
1949-03-31,132
1949-04-30,129
1949-05-31,121
...,...
1960-08-31,606
1960-09-30,508
1960-10-31,461
1960-11-30,390


In [12]:
df.columns = ['values']
series = df['values'].values
series

array([112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118, 115,
       126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140, 145, 150,
       178, 163, 172, 178, 199, 199, 184, 162, 146, 166, 171, 180, 193,
       181, 183, 218, 230, 242, 209, 191, 172, 194, 196, 196, 236, 235,
       229, 243, 264, 272, 237, 211, 180, 201, 204, 188, 235, 227, 234,
       264, 302, 293, 259, 229, 203, 229, 242, 233, 267, 269, 270, 315,
       364, 347, 312, 274, 237, 278, 284, 277, 317, 313, 318, 374, 413,
       405, 355, 306, 271, 306, 315, 301, 356, 348, 355, 422, 465, 467,
       404, 347, 305, 336, 340, 318, 362, 348, 363, 435, 491, 505, 404,
       359, 310, 337, 360, 342, 406, 396, 420, 472, 548, 559, 463, 407,
       362, 405, 417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390,
       432])

#### Prepare Dataframe for Forecasts

In [14]:
HORIZON = 12*4
start = df.index.max() + pd.DateOffset(months=1)
end = start + pd.DateOffset(months=HORIZON-1)
start, end

(Timestamp('1961-01-31 00:00:00'), Timestamp('1964-12-31 00:00:00'))

In [16]:
idx = pd.date_range(start=start,end=end,freq="ME",name='date')
idx

DatetimeIndex(['1961-01-31', '1961-02-28', '1961-03-31', '1961-04-30',
               '1961-05-31', '1961-06-30', '1961-07-31', '1961-08-31',
               '1961-09-30', '1961-10-31', '1961-11-30', '1961-12-31',
               '1962-01-31', '1962-02-28', '1962-03-31', '1962-04-30',
               '1962-05-31', '1962-06-30', '1962-07-31', '1962-08-31',
               '1962-09-30', '1962-10-31', '1962-11-30', '1962-12-31',
               '1963-01-31', '1963-02-28', '1963-03-31', '1963-04-30',
               '1963-05-31', '1963-06-30', '1963-07-31', '1963-08-31',
               '1963-09-30', '1963-10-31', '1963-11-30', '1963-12-31',
               '1964-01-31', '1964-02-29', '1964-03-31', '1964-04-30',
               '1964-05-31', '1964-06-30', '1964-07-31', '1964-08-31',
               '1964-09-30', '1964-10-31', '1964-11-30', '1964-12-31'],
              dtype='datetime64[ns]', name='date', freq='ME')

In [17]:
df_forecast = pd.DataFrame(index=idx)
df_forecast

""
date
1961-01-31
1961-02-28
1961-03-31
1961-04-30
1961-05-31
1961-06-30
1961-07-31
1961-08-31
1961-09-30
